# Predicting Airlines Ticket Price using Machine Learning

In [1]:
#import all necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#import data

train_data = pd.read_excel('Data_Train.xlsx')

In [ ]:
train_data.head()